In [ ]:
import math

def solve_star_labeling(m_in=None, n_in=None):
    print("--- Algorithm 1: Perfect Star-Labeling (S_m,n) [Fixed] ---")

    # 1. Input Validation
    if m_in is not None and n_in is not None:
        m, n = m_in, n_in
        print(f"Using provided inputs: m={m}, n={n}")
    else:
        try:
            m = int(input("Enter positive integer m (>= 2): "))
            n = int(input("Enter positive integer n (>= 2): "))
            if m < 2 or n < 2:
                print("Error: Both m and n must be >= 2.")
                return
        except ValueError:
            print("Error: Please enter valid integers.")
            return

    # 2. Initialization
    V = m * n + 1
    target_k = math.ceil(V / 2)

    # We want labels to span from 1 to target_k evenly.
    # The first label is 1, so we have (target_k - 1) distance to cover over (m - 1) steps.
    Diff = (target_k - 1) / (m - 1)

    print(f"\n[Topology] V: {V} (m={m}, n={n})")
    print(f"[Target]   Perfect k-max = ceil(V/2) = {target_k}")
    print(f"[Params]   Diff: {Diff:.4f}, Centroid Label (L0): 1")

    # 3. Process Level 1 Vertices (Array L1)
    L1 = []

    # Initialize accumulator to 1 (matching the first label)
    L1.append({"index": 1, "accumulator": 1, "label": 1, "weight": 2})

    for i in range(2, m + 1):
        prev = L1[-1]

        # Add Diff to the previous ACCUMULATOR (not 0)
        new_acc = prev["accumulator"] + Diff

        # Rounding logic: creating distinct steps.
        # Using simple floor here usually works if Diff is calculated correctly.
        calc_label = math.floor(new_acc)

        # Ensure strict monotonicity (Label must be > prev label)
        new_label = max(calc_label, prev["label"] + 1)

        # Satefy check: Try not to exceed target_k if possible (optional heuristic)
        # But for valid irregularity, we simply ensure it increases.

        new_weight = new_label + 1

        L1.append({"index": i, "accumulator": new_acc, "label": new_label, "weight": new_weight})

    # 4. Process Level 2 Vertices (Pendants) - Array L2
    L2 = []
    used_weights = set(item["weight"] for item in L1)
    h = 1
    total_pendants = m * (n - 1)

    for wt in range(3, V + 2):
        if h > total_pendants:
            break

        if wt not in used_weights:
            # Distribute pendants to parents evenly
            parent_idx_calc = math.floor((h + n - 2) / (n - 1))

            # Safety check for index
            if 1 <= parent_idx_calc <= m:
                parent_label = L1[parent_idx_calc - 1]["label"]

                # Calculate required label for this vertex to achieve the weight
                vertex_label = wt - parent_label

                L2.append({"h_index": h, "parent_arm_index": parent_idx_calc,
                    "weight": wt, "label": vertex_label})
                h += 1

    # 5. Output Display
    print("\nArray L1 (Level 1 - Connected to Centroid):")
    print(f"{'Index':<10} | {'Label':<10} | {'Edge Weight (1+L)':<20}")
    print("-" * 45)
    for item in L1:
        print(f"{item['index']:<10} | {item['label']:<10} | {item['weight']:<20}")

    print("\nArray L2 (Level 2 - Pendants):")
    print(f"{'No.':<6} | {'Parent Arm':<12} | {'Label':<10} | {'Edge Weight':<15}")
    print("-" * 50)
    for item in L2:
        print(f"{item['h_index']:<6} | {item['parent_arm_index']:<12} | {item['label']:<10} | {item['weight']:<15}")

    # 6. Verification
    verify_results(m, n, L1, L2, V, target_k)

def verify_results(m, n, L1, L2, V, target_k):
    print("\n--- Verification Report ---")
    weights = []
    max_label = 0
    all_labels = set()

    # Check L1
    for item in L1:
        weights.append(item['weight'])
        all_labels.add(item['label'])
        if item['label'] > max_label: max_label = item['label']

    # Check L2
    for item in L2:
        weights.append(item['weight'])
        all_labels.add(item['label'])
        if item['label'] > max_label: max_label = item['label']

    # Centroid label is always 1
    all_labels.add(1)

    unique_weights = set(weights)
    num_edges = len(weights)
    expected_edges = V - 1

    print(f"Target k (Perfect Limit): {target_k}")
    print(f"Actual Max Label Used:    {max_label}")
    print(f"Unique Edge Weights:      {len(unique_weights)} / {expected_edges}")

    if len(unique_weights) == expected_edges:
        print(">> [PASS] Edge Weights are Unique (Irregular).")
    else:
        print(">> [FAIL] Duplicate weights or missing edges.")

    if max_label <= target_k:
        print(f">> [PASS] PERFECT LABELING ({max_label} <= {target_k})")
    else:
        print(f">> [INFO] Imperfect. Diff: {max_label - target_k}")

if __name__ == "__main__":
    solve_star_labeling()



--- Algorithm 1: Perfect Star-Labeling (S_m,n) [Fixed] ---
